In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
#from datetime import datetime, timedelta
import backtrader as bt
import quantstats
from collections import defaultdict

//anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## Dual SMA strategy combining short-long and extension signals 
Only works for single-ticker portfolio

In [3]:
class SMA_dual_1ticker(bt.Strategy): 
    # Strategy parameters
    params = (('wshort',15),('wlong',45),('shortlongthresh',0.05),('wext',30),('extthresh',0.25))

    def __init__(self):
         
        #I think I need to change datas[0] to get all the data streams (ie multiple tickers)
        self.dataclose = self.datas[0].close
        
        #print(self.datas[0]) #datas[i] contains the data for the ith ticker
        
        # Order variable will contain ongoing order details/status
        self.order = None

        #Get indicators
        self.short_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.wshort, plotname='Short SMA')
        self.long_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.wlong, plotname='Long SMA')
        self.ext_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.wext, plotname='Extension SMA')
        self.sma200 = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=200, plotname='200-day SMA')
        #self.std = bt.indicators.StandardDeviation(self.data, period = self.params.wext)
        
        #Calculate signals
        self.extension = (self.dataclose - self.ext_sma)/ self.ext_sma
        self.shortlongratio = self.short_sma/self.long_sma
            
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}') # Comment this line when running optimization
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # An active Buy/Sell order has been submitted/accepted - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                #self.log(f'BUY EXECUTED, {order.executed.price:.2f}, {order.data._name}')
                self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Reset orders
        self.order = None
            
    def next(self):
        # Check for open orders
        if self.order:
            return

        # Check if we are in the market
        if self.position.size == 0:
            # We are not in the market, look for a signal to BUY
            #if self.dataclose > self.sma200: #Only buy if the price is above the 200-day SMA
            if self.extension < -self.params.extthresh and self.shortlongratio < 1-self.params.shortlongthresh:
                self.log(f'BUY CREATE {self.dataclose[0]:2f}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
        elif self.position.size > 0:
            # We are in the market, look for a signal to SELL
            if self.extension > -self.params.extthresh or self.shortlongratio > 1:
                self.log(f'SELL CREATE {self.dataclose[0]:2f}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
            
                
            #Not sure where to put the CLOSE CREATE to ensure we close any positions at the end of a backtest
        #else:
            # We are already in the market, look for a signal to CLOSE trades
            #if len(self) >= (self.bar_executed + 5):
                #self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
                #self.order = self.close()

## Dual SMA strategy combining short-long and extension signals 
Works for portfolio containing multiple tickers

To add in updated strategy:
    - limit-order/take-profit
    - better sizer (kelly criterion?)
    - try just long-short sma and extension sma separately

In [11]:
class SMAdual_basic(bt.Strategy):

    # Strategy parameters
    params = (('wshort',15),('wlong',45),('shortlongthresh',0.05),('wext',30),('extthresh',0.25),('sma200_switch',False),('verbose',False))

    def __init__(self):
        #Initialize signal storage
        self.shortlongratios = []
        self.extensions = []
        self.sma_200s = []
         
        #For each ticker
        for d in self.datas: 
            #Calculate indicatora for ticker d
            short_sma = bt.ind.SMA(d, period=self.params.wshort)
            long_sma = bt.ind.SMA(d, period=self.params.wlong)
            ext_sma = bt.ind.SMA(d, period=self.params.wext)
            sma_200 = bt.ind.SMA(d, period=200)
            
            #Calculate signals for ticker d
            shortlongratio = short_sma/long_sma
            extension = (d.close - ext_sma)/ext_sma
            
            #Append signals for ticker d to storage
            self.shortlongratios.append(shortlongratio)
            self.extensions.append(extension)
            self.sma_200s.append(sma_200)
    
    def log(self, txt, dt=None, verbose=False):
        dt = dt or self.datas[0].datetime.date(0)
        if verbose:
            print(f'{dt.isoformat()} {txt}') # Comment this line when running optimization
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # An active Buy/Sell order has been submitted/accepted - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                #self.log(f'BUY EXECUTED, {order.executed.price:.2f}, {order.data._name}')
                self.log(f'{order.data._name} BUY EXECUTED, {order.executed.price:.2f}', verbose=self.params.verbose)
            elif order.issell():
                self.log(f'{order.data._name} SELL EXECUTED, {order.executed.price:.2f}', verbose=self.params.verbose)
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected', verbose=self.params.verbose)

        # Reset orders
        self.order = None
 
    def next(self):
        #For each ticker
        for i, d in enumerate(self.datas):
            #if position is zero (we don't hold ticker d)
            if not self.getposition(d).size:
                #Only consider buying if the price is above the 200-day SMA
                if (d.close > self.sma_200s[i]) or (not self.params.sma200_switch): 
                    #If crossover signal is positive
                    if (self.extensions[i] < -self.params.extthresh) and (self.shortlongratios[i] < 1-self.params.shortlongthresh):
                        #Buy ticker d
                        self.buy(data = d)
            #only reached if position is not zero (we hold ticker d):
                #if crossover signal is negative
            elif (self.extensions[i] > -self.params.extthresh) and (self.shortlongratios[i] > 1): 
                #Sell ticker d
                self.close(data = d)
    
    def stop(self):
        self.log('Short SMA %2d, Long SMA %2d, Extension SMA %2d, Balance %.2f' %
                 (self.params.wshort, self.params.wlong, self.params.wext, self.broker.getvalue()), verbose=self.params.verbose)

## Tutorial SMA Crossover Strategy for multiple tickers

In [5]:
class MaCrossStrategy(bt.Strategy):

    params = (('fast_length', 5),('slow_length', 25))
     
    def __init__(self):
        #Initialize signal storage
        self.crossovers = []
         
        #For each ticker
        for d in self.datas: 
            #Calculate indicators for ticker d
            ma_fast = bt.ind.SMA(d, period = self.params.fast_length)
            ma_slow = bt.ind.SMA(d, period = self.params.slow_length)
            
            #Calculate signals for ticker d
            crossover = bt.ind.CrossOver(ma_fast, ma_slow)
            
            #Append signals for ticker d to storage
            self.crossovers.append(crossover)
 
    def next(self):
        #For each ticker
        for i, d in enumerate(self.datas):
            #if position is zero (we don't hold ticker d)
            if not self.getposition(d).size:
                #If crossover signal is positive
                if self.crossovers[i] > 0:
                    #Buy ticker d
                    self.buy(data = d)
            #only reached if position is not zero (we hold ticker d):
                #if crossover signal is negative
            elif self.crossovers[i] < 0: 
                #Sell ticker d
                self.close(data = d)